In [ ]:
import ollama
import chromadb
import warnings
import textwrap

# --- 1. SETUP AND CONNECTIONS ---

# Connect to your database
db_path = "chroma_db"
client = chromadb.PersistentClient(path=db_path)
collection_name = "fifth_standard_books"
collection = client.get_collection(name=collection_name)
print(f"✅ Connected to ChromaDB. Collection has {collection.count()} chunks.")

# --- 2. DEFINE THE OLLAMA MODEL TO USE ---
# You can easily switch between your downloaded models here
# OLLAMA_MODEL = 'gemma3:4b' 
OLLAMA_MODEL = 'llama3.2:latest' 
print(f"✅ Using local Ollama model: '{OLLAMA_MODEL}'")


# --- 3. THE "PERFECTED" RAG BRAIN (using Ollama) ---

# Store user info and conversation history
user_profile = {}
conversation_history = []

def get_personalized_response(user_query):
    global conversation_history
    
    # Retrieve relevant chunks from ChromaDB
    results = collection.query(query_texts=[user_query], n_results=3)
    context = "\n\n---\n\n".join(results['documents'][0])
    
    # --- DYNAMIC PROMPT ENGINEERING ---
    persona_prompt = ""
    if user_profile:
        name = user_profile.get('name', 'student')
        standard = user_profile.get('standard', 5)
        
        if standard == 5:
            persona_prompt = f"You are a friendly and fun chatbot friend for a {standard}th grade student named {name}. Your goal is to make learning feel like a game. Explain concepts using simple stories, fun examples, or by creating a mini-puzzle. Always be encouraging and positive. Address the user by their name."
        else:
            persona_prompt = f"You are a helpful and knowledgeable tutor for a {standard}th grade student named {name}. Explain concepts clearly and concisely."
            
    # Build the conversation history for the model
    messages = []
    # Add the main system prompt (the persona)
    messages.append({'role': 'system', 'content': persona_prompt})
    
    # Add the last few turns of history
    for turn in conversation_history[-2:]:
        messages.append({'role': 'user', 'content': turn['user']})
        messages.append({'role': 'assistant', 'content': turn['bot']})
        
    # Add the final user prompt with RAG context
    final_prompt = f"""Use ONLY the following context from the textbook to answer my question.
    
    CONTEXT:
    {context}
    
    QUESTION:
    {user_query}
    """
    messages.append({'role': 'user', 'content': final_prompt})
    
    # --- GENERATION (The new Ollama call) ---
    try:
        response = ollama.chat(
            model=OLLAMA_MODEL,
            messages=messages
        )
        answer = response['message']['content']
        # Add this turn to our conversation history
        conversation_history.append({"user": user_query, "bot": answer})
        return answer
    except Exception as e:
        return f"An error occurred with Ollama: {e}"

# --- 4. THE INTERACTIVE CHAT INTERFACE (No changes here) ---
def start_personalized_chat():
    global user_profile
    print("\n" + "="*50)
    print("🤖 **Welcome to your Personal Learning Assistant!**")
    name = input("   What's your name? ")
    age = input("   How old are you? ")
    standard = int(input("   What grade (standard) are you in? "))
    user_profile = {"name": name, "age": age, "standard": standard}
    print(f"\nAwesome to meet you, {name}! Let's make learning fun. Ask me anything!")
    print("   (Type 'quit' or 'exit' to end our chat)")
    print("="*50)
    
    while True:
        user_input = input(f"{name}: ")
        if user_input.lower() in ["quit", "exit"]:
            print("\n🤖 It was fun learning with you! Goodbye!")
            break
        
        response = get_personalized_response(user_input)
        wrapped_response = textwrap.fill(response, width=80)
        print(f"\n🤖 Tutor:\n{wrapped_response}\n" + "-"*50)

# --- START THE CHATBOT ---
start_personalized_chat()

✅ Connected to ChromaDB. Collection has 1918 chunks.
✅ Using local Ollama model: 'llama3.2:latest'

🤖 **Welcome to your Personal Learning Assistant!**

Awesome to meet you, Sajith! Let's make learning fun. Ask me anything!
   (Type 'quit' or 'exit' to end our chat)


C:\Users\sajit\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [02:33<00:00, 542kiB/s] 



🤖 Tutor:
Hi Sajith!   Let's talk about trigonometry.  From what we've learned earlier
today, you made an angle measuring tool using tracing paper. This tool helped us
measure angles.  Trigonometry is actually the study of triangles, especially
right triangles (triangles with one 90-degree angle). It involves understanding
the relationships between the sides and angles of these triangles.  Think of it
like this: imagine you have a triangle with two shorter sides (called legs) and
a longer side opposite one of the angles. Trigonometry helps us figure out the
length of that long side, or other unknowns in the triangle, using the known
angles and side lengths.  In your angle measuring tool, when you placed the
straw at the center and measured turns with respect to the starting point, you
were actually getting a sense of the relationships between the angles and sides
of the triangle. That's basically what trigonometry is all about!  Does that
make sense?
-----------------------------------